In [24]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans

In [25]:
all_data = loadmat('data/sample_1.mat')
sample_data = np.array(all_data['data'])
spike_times = np.array(all_data['spike_times'])

In [ ]:
seconds = 1.5
sr = 24000

data = sample_data[0][:round(seconds*sr)]
plt.figure(figsize=(25,6))
plt.plot(np.linspace(0, seconds, round(seconds*sr)), data)
plt.xlim(0, seconds)

spike_times_reconstructed = np.where(data >= 25)[0]
spike_times_start_only = []
spike_times_start_only.append(spike_times_reconstructed[0])
spike_times_start_only.extend(spike_times_reconstructed[np.where(np.diff(spike_times_reconstructed) > 10)[0]+1])

for i, spike in enumerate(spike_times_start_only):
    # plt.axvline(spike/sr - .0008, c='red', alpha=0.5)
    plt.axvspan(spike/sr - .0008, spike/sr + .001, facecolor='r', alpha=0.2)
    plt.text(spike/sr - 0.006, 110 if i%2 == 0 else 100, i+1, c='r')
    
for i, spike in enumerate(spike_times[0][0][0][:round(seconds*sr)]/sr):
    if round(spike*24000) > round(seconds*24000): break
    plt.axvline(spike, c='k', alpha=1)
    plt.text(spike - 0.006, 85 if i%2 == 0 else 75, i+1, c='b')
    
plt.show()

In [ ]:
number_of_spikes = len(spike_times_start_only)

ymax = 0
ymin = 0
spikes = []
for spike in spike_times_start_only:
    start = round(spike - .0008 * sr)
    end = round(spike + .001 * sr)
    if np.max(data[start:end]) > ymax: ymax = np.max(data[start:end])
    if np.min(data[start:end]) < ymin: ymin = np.min(data[start:end])
    spikes.append(data[start:end])

inertias = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, n_init='auto')
    kmeans.fit(spikes)
    inertias.append(kmeans.inertia_)

plt.plot(range(1, 11), inertias, marker='.')
plt.show()


In [ ]:
plt.figure(figsize=(30,10))

kmeans_final = KMeans(n_init='auto', n_clusters=3)
classes = kmeans_final.fit_predict(spikes)
colors = ['r', 'g', 'b']

for i, spike in enumerate(spike_times_start_only):
    start = round(spike - .0008 * sr)
    end = round(spike + .001 * sr)
    plt.subplot(5, math.ceil(number_of_spikes/5), i+1)
    plt.grid()
    plt.ylim(ymin, ymax)
    plt.plot(data[start:end], c=colors[classes[i]])
plt.show()